# VacationPy
----



In [138]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [139]:
# Importing cities data
cities_data_to_load = "../WeatherPy/cities.csv"
cities_data = pd.read_csv(cities_data_to_load)
cities_data.drop('Unnamed: 0', inplace=True, axis=1)
cities_data

,City,Cloudiness,Country,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Tura,90,IN,96,25.5198,90.2201,71.91,4.07
1,Kyshtovka,100,RU,81,56.5550,76.6271,47.12,12.86
2,Mount Gambier,100,AU,54,-37.8333,140.7667,56.39,12.17
3,New Norfolk,8,AU,74,-42.7826,147.0587,46.71,1.81
4,Rikitea,0,PF,74,-23.1203,-134.9692,71.67,10.47
...,...,...,...,...,...,...,...,...
560,Kavaratti,100,IN,80,10.5669,72.6420,80.69,9.15
561,Gondanglegi,100,ID,90,-7.7479,112.0132,76.15,10.83
562,Sechura,30,PE,60,-5.5569,-80.8222,74.88,14.74
563,Sidney,75,AU,87,-33.8679,151.2073,57.27,5.75


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [140]:
# configuring gmaps
gmaps.configure(api_key=g_key)

locations = cities_data[['Lat', 'Lng']]
humidity = cities_data['Humidity']
max_humidity = humidity.max()

fig = gmaps.figure(center=(0,0), zoom_level=1)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

Ideal weather conditions:
- A max temperature lower than 80 degrees but higher than 70
- Wind speed less than 10 mph
- Zero cloudiness
- Humidity is under 30

In [141]:
ideal_weather = cities_data.loc[(cities_data['Wind Speed'] <= 10)][(cities_data['Cloudiness'] == 0)][(cities_data['Max Temp'] < 80)][(cities_data['Max Temp'] > 70)][(cities_data['Humidity'] < 30)].dropna()
ideal_weather

C:\Users\haldu\AppData\Local\Temp\ipykernel_7840\2792009468.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  ideal_weather = cities_data.loc[(cities_data['Wind Speed'] <= 10)][(cities_data['Cloudiness'] == 0)][(cities_data['Max Temp'] < 80)][(cities_data['Max Temp'] > 70)][(cities_data['Humidity'] < 30)].dropna()


,City,Cloudiness,Country,Humidity,Lat,Lng,Max Temp,Wind Speed
83,Balkanabat,0,TM,15,39.5108,54.3671,73.29,9.89
195,Susanville,0,US,26,40.4163,-120.6530,75.13,1.01
226,Nokaneng,0,BW,15,-19.6667,22.2667,75.65,6.31
307,Belle Fourche,0,US,20,44.6714,-103.8522,77.50,2.82
390,Fallon,0,US,23,39.4735,-118.7774,77.45,4.61
430,Sun Valley,0,US,26,39.5963,-119.7760,77.02,5.75
448,Tartagal,0,AR,13,-22.5164,-63.8013,76.68,6.26
483,Neuquen,0,AR,13,-38.9516,-68.0591,75.09,2.30


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [142]:
# adding a column called 'Hotel Name'
hotel_df = ideal_weather
hotel_df['Hotel Name'] = ''
hotel_df

,City,Cloudiness,Country,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
83,Balkanabat,0,TM,15,39.5108,54.3671,73.29,9.89,
195,Susanville,0,US,26,40.4163,-120.6530,75.13,1.01,
226,Nokaneng,0,BW,15,-19.6667,22.2667,75.65,6.31,
307,Belle Fourche,0,US,20,44.6714,-103.8522,77.50,2.82,
390,Fallon,0,US,23,39.4735,-118.7774,77.45,4.61,
430,Sun Valley,0,US,26,39.5963,-119.7760,77.02,5.75,
448,Tartagal,0,AR,13,-22.5164,-63.8013,76.68,6.26,
483,Neuquen,0,AR,13,-38.9516,-68.0591,75.09,2.30,


In [143]:
# reindexing dataframe
hotel_df = hotel_df.reset_index()
del hotel_df['index']
hotel_df


,City,Cloudiness,Country,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,Balkanabat,0,TM,15,39.5108,54.3671,73.29,9.89,
1,Susanville,0,US,26,40.4163,-120.6530,75.13,1.01,
2,Nokaneng,0,BW,15,-19.6667,22.2667,75.65,6.31,
3,Belle Fourche,0,US,20,44.6714,-103.8522,77.50,2.82,
4,Fallon,0,US,23,39.4735,-118.7774,77.45,4.61,
5,Sun Valley,0,US,26,39.5963,-119.7760,77.02,5.75,
6,Tartagal,0,AR,13,-22.5164,-63.8013,76.68,6.26,
7,Neuquen,0,AR,13,-38.9516,-68.0591,75.09,2.30,


In [144]:

params = {
     'radius': 5000,
     'types': 'lodging',
     'key': g_key,
}

for index, row in hotel_df.iterrows():
    lat = row['Lat']
    lon = row['Lng']
    city_name = row['City']

    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

    params['location'] = f'{lat},{lon}'
    print(f'Retrieving Results for Index {index} - {city_name}')
    response = requests.get(base_url, params=params)
    response = response.json()

    try:
     
     print(f'Closest hotel in {city_name} is {response["results"][0]["name"]}.')
     hotel_df.loc[index,'Hotel Name'] = response['results'][0]['name']
     print('-'*30)

    except:
          print('Missing field')
     


Retrieving Results for Index 0 - Balkanabat
Closest hotel in Balkanabat is Nebitchi Hotel.
------------------------------
Retrieving Results for Index 1 - Susanville
Closest hotel in Susanville is Comfort Inn & Suites.
------------------------------
Retrieving Results for Index 2 - Nokaneng
Missing field
Retrieving Results for Index 3 - Belle Fourche
Closest hotel in Belle Fourche is Super 8 by Wyndham Belle Fourche SD.
------------------------------
Retrieving Results for Index 4 - Fallon
Closest hotel in Fallon is Best Western Fallon Inn & Suites.
------------------------------
Retrieving Results for Index 5 - Sun Valley
Closest hotel in Sun Valley is Panther Valley RV Park.
------------------------------
Retrieving Results for Index 6 - Tartagal
Closest hotel in Tartagal is Almuñecar Hotel.
------------------------------
Retrieving Results for Index 7 - Neuquen
Closest hotel in Neuquen is Hotel del Comahue.
------------------------------


In [145]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [147]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Display figure
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))